## AIHub Json Preprocessing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [1]:
import re
import os
import kss
import ray
import json
import time
from time import sleep
from tqdm import tqdm
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [19]:
pwd

'd:\\AIHUB'

### Basic Function

In [2]:
def sorted_list(path_list):
    
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [3]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [4]:
def json_file_path_list(path_list):
    
    json_file_path = [glob(i, recursive = True) for i in path_list][0]
    json_file_path = sorted_list(json_file_path)
    
    return json_file_path

In [5]:
def train_valid_json_file_path_list(path_list):

  train_json_file_path, valid_json_file_path = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_json_file_path = sorted_list(train_json_file_path)
  valid_json_file_path = sorted_list(valid_json_file_path)
    
  return train_json_file_path, valid_json_file_path

In [6]:
def txt_file_path_list(json_file_path, folder_corpus_type_path):
  
  txt_file_path = [folder_corpus_type_path + str(i) + ".txt"
                              for i in range(len(json_file_path))]
    
  return txt_file_path

In [7]:
def make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list):

    train_json_file_path, valid_json_file_path = train_valid_json_file_path_list(json_path_list)
    
    the_number_of_train_json_file  = len(train_json_file_path)
    the_number_of_valid_json_file  = len(valid_json_file_path)
    the_number_of_json_file = the_number_of_train_json_file + the_number_of_valid_json_file
    print("The number of train json file:", the_number_of_train_json_file)
    print("The number of valid json file:", the_number_of_valid_json_file)
    print("The number of json file:", the_number_of_json_file)
    
    train_txt_file_path = txt_file_path_list(train_json_file_path, txt_path_list[0])
    valid_txt_file_path = txt_file_path_list(valid_json_file_path, txt_path_list[1])

    return train_json_file_path, valid_json_file_path, train_txt_file_path, valid_txt_file_path

In [8]:
def make_json_txt_file_path_list(json_path_list, txt_path_list):

    json_file_path = json_file_path_list(json_path_list)
    
    the_number_of_json_file = len(json_file_path) 
    print("The number of json file:", the_number_of_json_file)
    
    txt_file_path = txt_file_path_list(json_file_path, txt_path_list[0])

    return json_file_path, txt_file_path

In [9]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, " " + text + " ")    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, " " + text + " ")    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    
        if source[0] == '"':
            del(source[0])
        elif source[-1] == '"':
            del(source[0])
        elif source[0] == '"' and source[-1] == '"':
            del(source[0])
            del(source[-1])
        # 문장의 시작과 끝이 따옴표("")이면 따옴표 제거
        
        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

### AIHUB 기계독해

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=89)

#### Convert JSON File to TXT File

In [10]:
json_path_list = ['AIHUB_기계독해'+ '/**/*.json']
txt_path_list =  ["exploration/machine_reading_pro/AIHUB_machine_reading_"]

In [11]:
json_file_list, txt_file_path_list = \
    make_json_txt_file_path_list(json_path_list, txt_path_list)

The number of file: 3


In [12]:
source_file_index_df = pd.DataFrame(json_file_list, columns=['source_file_name'])
source_file_index_df.to_excel("source_file_index/machine_reading_source_file_index.xlsx", index=False)

In [13]:
def make_source_list(json_sample):

    source_df = pd.DataFrame(json_sample['data'])
    source_dict = dict(source_df['paragraphs'].explode())
    source_json = pd.json_normalize(source_dict)
    source_list = list(source_json.filter(regex='context').values[0])

    return source_list

In [14]:
def count_number_of_txt_file_with_batch_list(source_file_list, batch_size):

    source_file_by_batch_df = pd.DataFrame({'File':[0], 'Length of Source List':[0],
                                        'The Number of TXT File':[0], 
                                        'Description':[0]})
                                            
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    
    for i in range(len(source_file_list)):    
        
        source_file = source_file_list[i]   

        with open(source_file, 'r', encoding='utf-8') as one_json_file:
            one_json_sample = json.load(one_json_file)

        source_list = make_source_list(one_json_sample)
        
        the_number_of_txt_file = ((len(source_list) // batch_size) + 1)

        if len(source_list) >= batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file, ""]
            the_number_of_txt_file_list.append(the_number_of_txt_file)
            the_number_of_total_txt_file  += the_number_of_txt_file

        elif len(source_list) < batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file,
                                              "not subject of batch. small source list."]
            the_number_of_txt_file_list.append(1)
            the_number_of_total_txt_file  += 1

    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)

    if 'rain' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/machine_reading_train.xlsx", index=False)
    elif 'alid' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/machine_reading_valid.xlsx", index=False)
    else:
         source_file_by_batch_df.to_excel("source_file_by_batch/machine_reading.xlsx", index=False)

    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [15]:
def write_jsontext_to_txt_file_with_batch_list(source_file_list,
                                    text_file_path_list,
                                    batch_size, the_number_of_txt_file_list):
  
  
  progress_length = sum(the_number_of_txt_file_list)
  print("[Size]")
  print("The number of preprocessing corpus: " + str(progress_length))
  print("\n[Order]")
  pbar = tqdm(range(progress_length))
  num = 0

  for i in range(len(source_file_list)):

    source_file = source_file_list[i]
    
    with open(source_file, 'r', encoding='utf-8') as one_json_file:
      one_json_sample = json.load(one_json_file)

    source_list = make_source_list(one_json_sample)
    
    n = batch_size
    source_batch_list = list(divide_source_file_list(source_list, n))
      
    for source_list in source_batch_list:
        with open(os.path.join('AIHUB_corpus/' + text_file_path_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
          fp.write("\n".join(source_list)) 
        num += 1  
        pbar.n += 1
        pbar.refresh()
        time.sleep(0.01)
  pbar.close()  

In [16]:
batch_size = 1000
the_number_of_txt_file, the_number_of_txt_file_list = count_number_of_txt_file_with_batch_list(json_file_list, batch_size)

Batch Size: 1000
The number of txt file: 104


In [18]:
source_file_by_batch_df = pd.read_excel('source_file_by_batch/machine_reading.xlsx', engine='openpyxl')  
source_file_by_batch_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_기계독해\ko_nia_normal_squad_all.json ~ AI...,47314,48,NaN
1,1,AIHUB_기계독해\ko_nia_clue0529_squad_all.json ~ ...,34500,35,NaN
2,2,AIHUB_기계독해\ko_nia_noanswer_squad_all.json ~ ...,20030,21,NaN


In [19]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(json_file_list, txt_file_path_list,
                batch_size, the_number_of_txt_file_list)

[Size]
The number of preprocessing corpus: 104

[Order]


100%|██████████| 104/104 [00:20<00:00,  5.14it/s]


#### Preprocess TXT File

In [16]:
def post_txt_file_path_list(corpus_path_list):
   
  post_corpus_path_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_path_list]

  return post_corpus_path_list

In [17]:
def make_pro_post_txt_file_path_list(pro_corpus_path):
    
    pro_total_corpus_path_list = glob(pro_corpus_path)
    pro_total_corpus_path_list = sorted_list(pro_total_corpus_path_list)
    post_total_corpus_path_list = post_txt_file_path_list(pro_total_corpus_path_list)

    return pro_total_corpus_path_list, post_total_corpus_path_list

In [18]:
pro_corpus_path = "AIHUB_corpus/exploration/machine_reading_pro/AIHUB_machine_reading_" + "*.txt"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_path_list(pro_corpus_path)

In [19]:
len(pro_total_corpus_path_list)

104

In [20]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 1:
           line_list.append(line)
        
for line in line_list:
    print(line, end="\n\n")

한국청소년단체협의회와 여성가족부는 22일부터 28일까지 서울과 충북 괴산에서 '국제청소년포럼'을 연다고 21일 밝혔다. 한국 미국 캐나다 호주 등 전 세계 32개국 75여명의 대학생, 청소년들이 모여 전 세계적 현안문제에 대한 대안과 해결책을 모색하는 자리다. 이번 포럼의 주제는 '청소년과 뉴미디어'다. 스마트폰 SNS 태블릿PC 등 새로운 커뮤니케이션 매체인 '뉴미디어'에 대한 성찰과 문제점에 대해 토론한다. 기조강연을 시작으로 국가별 주제관련 사례발표, 그룹 토론 및 전체총회, '청소년선언문' 작성 및 채택 등 다양한 프로그램을 운영한다. 개회식은 22일 서울 방화동에 있는 국제청소년센터 국제회의장에서 한다. 전 세계 32개국 대학생ㆍ청소년 참가자와 전국의 청소년기관단체장과 청소년지도자 여성가족부 주한외교사절 등 100여명이 참석할 예정이다. 23일에는 유엔미래포럼 박영숙 대표가 '뉴미디어의 균형 있는 발전을 위한 청소년의 역할'에 대해 기조강연을 한다. 뉴미디어의 올바른 활용방안과 청소년문화의 형성에 대해 설명할 계획이다. 27일 폐회식에서는 '청소년선언문'을 채택한다. 선언문에는 전 세계적으로 뉴미디어의 바람직한 발전을 촉구하며 각국 청년들이 함께 실천할 수 있는 내용 등이 담길 예정이다. 한국청소년단체협의회는 포럼이 끝난 뒤 UN 등 국제기구와 참가자 각국 정부 등 국제사회에 선언문을 전달할 예정이다.



In [21]:
line_list = []
line_num = 0
with open(pro_total_corpus_path_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 1:  
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

한국청소년단체협의회와 여성가족부는 22일부터 28일까지 서울과 충북 괴산에서  국제청소년포럼 을 연다고 21일 밝혔다.

한국 미국 캐나다 호주 등 전 세계 32개국 75여명의 대학생, 청소년들이 모여 전 세계적 현안문제에 대한 대안과 해결책을 모색하는 자리다.

스마트폰 SNS 태블릿PC 등 새로운 커뮤니케이션 매체인  뉴미디어 에 대한 성찰과 문제점에 대해 토론한다.

기조강연을 시작으로 국가별 주제관련 사례발표, 그룹 토론 및 전체총회,  청소년선언문  작성 및 채택 등 다양한 프로그램을 운영한다.

개회식은 22일 서울 방화동에 있는 국제청소년센터 국제회의장에서 한다.

전 세계 32개국 대학생 청소년 참가자와 전국의 청소년기관단체장과 청소년지도자 여성가족부 주한외교사절 등 100여명이 참석할 예정이다.

23일에는 유엔미래포럼 박영숙 대표가  뉴미디어의 균형 있는 발전을 위한 청소년의 역할 에 대해 기조강연을 한다.

뉴미디어의 올바른 활용방안과 청소년문화의 형성에 대해 설명할 계획이다.

선언문에는 전 세계적으로 뉴미디어의 바람직한 발전을 촉구하며 각국 청년들이 함께 실천할 수 있는 내용 등이 담길 예정이다.

한국청소년단체협의회는 포럼이 끝난 뒤 UN 등 국제기구와 참가자 각국 정부 등 국제사회에 선언문을 전달할 예정이다.



In [22]:
ray.init(num_cpus = 4)

@ray.remote
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, " " + text + " ")    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, " " + text + " ")    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외

            if source[0] == '"':
                del(source[0])
            elif source[-1] == '"':
                del(source[0])
            elif source[0] == '"' and source[-1] == '"':
                del(source[0])
                del(source[-1])
            # 문장의 시작과 끝이 따옴표("")이면 따옴표 제거

            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

2023-07-11 10:42:24,945	INFO worker.py:1625 -- Started a local Ray instance.


In [23]:
def preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list):
    
    progress_length = len(pro_total_corpus_path_list)
    print("[Size]")
    print("The number of preprocessing corpus: " + str(progress_length))
    print("\n[Order]")
    pbar = tqdm(range(progress_length))
    process_num = 10    

    for pro, post in zip(pro_total_corpus_path_list, post_total_corpus_path_list):

        sentence_list = []

        with open(pro, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines() 
            nested_lines_num = len(lines) // process_num
            for i in range(nested_lines_num - 1):
                start_line = process_num * i
                end_line = process_num * (i+1)
                futures = [formal_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
                results = ray.get(futures)

                if i == nested_lines_num - 2:
                    futures = [formal_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)

                sentences = list(chain.from_iterable(results))
                sentence_list.append(sentences)

        sentence_list = list(chain.from_iterable(sentence_list))

        with open(post, 'a', encoding='utf-8') as fp:
            fp.write("\n".join(sentence_list))

        pbar.n += 1
        pbar.refresh()
        time.sleep(0.01)

    pbar.close() 

In [24]:
preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list)

[Size]
The number of preprocessing corpus: 104

[Order]


 96%|█████████▌| 100/104 [1:16:56<03:04, 46.16s/it](formal_preprocessing_text pid=19244) [Korean Sentence Splitter]: Too long text! turn off quotes calibration!
(formal_preprocessing_text pid=18688) [Korean Sentence Splitter]: Too long text! turn off quotes calibration!
100%|██████████| 104/104 [1:18:44<00:00, 45.43s/it]


In [223]:
ray.shutdown()

In [ ]:
def merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path,
                                      deduplicate_corpus_path):
    
    corpus_list = glob(preprocessing_corpus_path)
    corpus_list = sorted_list(corpus_list)
    
    with open(merge_corpus_path, 'w', encoding='utf-8') as f:
        for corpus in corpus_list:
            with open(corpus, encoding='utf-8') as text:
                for line in text:
                    f.write(line)
                    
    with open(deduplicate_corpus_path, 'w', encoding='utf-8') as f1:
        with open(merge_corpus_path, encoding='utf-8') as f2:
            lines = f2.read().splitlines()
            single_sentence_dict = dict.fromkeys(lines)
            single_sentence_list = list(single_sentence_dict)
            f1.write("\n".join(single_sentence_list))                

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/machine_reading_post/AIHUB_machine_reading_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_machine_reading.txt'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_machine_reading.txt'

In [ ]:
merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path, 
                                  deduplicate_corpus_path)